### Задание  
Разобраться с моделькой перевода как она устроена  
запустить для перевода с русского на английский (при желании можно взять другие пары языков) два варианта с вниманием и без внимания  
оценить качество насколько корректно переводит (для теста отобрать примеры с увеличением длины текста) (так как оценка визуальная достаточно 20-ти примеров в тестовой выборке)

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

### Чтение файла и подготовка датасета

In [2]:
path_to_file = "./content/rus-eng/rus.txt"

In [3]:
def preprocess_sentence(w):
    w = w.lower().strip()
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)
    w = w.strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    
    return w

In [4]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [5]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]
    
    return zip(*word_pairs)

In [6]:
en, ru = create_dataset(path_to_file, None)
print(en[0])
print(ru[0])

<start> go . <end>
<start> марш ! <end>


In [7]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='')
    lang_tokenizer.fit_on_texts(lang)
    
    tensor = lang_tokenizer.texts_to_sequences(lang)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post')
    
    return tensor, lang_tokenizer

In [8]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)
    
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
    
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [9]:
len(en), len(ru)

(444587, 444587)

In [10]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [11]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor,
                                                                                                target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [12]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [13]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

### Модель без механизма внимания

#### Энкодер

In [14]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=False,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
    
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

#### Декодер

In [15]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x, initial_state=hidden)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)
        
        return x, state

#### Модель

In [16]:
class MyTranslator():
    def __init__(self, dataset, vocab_inp_size, vocab_tar_size,
                 embedding_dim, units,
                 batch_size, steps_per_epoch, checkpoint_dir):
        self.dataset = dataset
        self.units = units
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch
        
        self.checkpoint_dir = checkpoint_dir
        self.checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
        
        self.encoder = Encoder(vocab_inp_size, embedding_dim, units, batch_size)
        self.decoder = Decoder(vocab_tar_size, embedding_dim, units, batch_size)
        
        self.optimizer = tf.keras.optimizers.Adam()
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        
        self.checkpoint = tf.train.Checkpoint(optimizer=self.optimizer,
                                              encoder=self.encoder,
                                              decoder=self.decoder)

    
    def loss_function(self, real, pred):
        mask = tf.math.logical_not(tf.math.equal(real, 0))
        loss_ = self.loss_object(real, pred)
    
        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ *= mask
    
        return tf.reduce_mean(loss_)
    
    
    def train_step(self, inp, targ, enc_hidden):
        loss = 0
    
        with tf.GradientTape() as tape:
            enc_hidden = self.encoder(inp, enc_hidden)
        
            dec_hidden = enc_hidden
        
            dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * self.batch_size, 1)
        
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden = self.decoder(dec_input, dec_hidden)
            
                loss += self.loss_function(targ[:, t], predictions)
            
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
    
        variables = self.encoder.trainable_variables + self.decoder.trainable_variables
    
        gradients = tape.gradient(loss, variables)
    
        self.optimizer.apply_gradients(zip(gradients, variables))
    
        return batch_loss
    
    
    def train_model(self, epochs):
        for epoch in range(epochs):
            start = time.time()
    
            enc_hidden = self.encoder.initialize_hidden_state()
            total_loss = 0

            for (batch, (inp, targ)) in enumerate(self.dataset.take(self.steps_per_epoch)):
                batch_loss = self.train_step(inp, targ, enc_hidden)
                total_loss += batch_loss
        
                if batch % 100 == 0:
                    print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                                 batch,
                                                                 batch_loss.numpy()))
            
            # saving (checkpoint) the model every 2 epochs
            if (epoch + 1) % 2 == 0:
                self.checkpoint.save(file_prefix = self.checkpoint_prefix)
        
            print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                            total_loss / self.steps_per_epoch))
            print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
            

    def evaluate(self, sentence):
        attention_plot = np.zeros((max_length_targ, max_length_inp))
    
        sentence = preprocess_sentence(sentence)
    
        inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
        inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                               maxlen=max_length_inp,
                                                               padding='post')
        inputs = tf.convert_to_tensor(inputs)
    
        result = ''

        hidden = [tf.zeros((1, self.units))]
        enc_hidden = self.encoder(inputs, hidden)

        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

        for t in range(max_length_targ):
            predictions, dec_hidden = self.decoder(dec_input, dec_hidden)
        
            predicted_id = tf.argmax(predictions[0]).numpy()
            result += targ_lang.index_word[predicted_id] + ' '
    
            if targ_lang.index_word[predicted_id] == '<end>':
                return result, sentence
    
            # the predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)
    
        return result, sentence
   

    def translate(self, sentence):
        result, sentence = self.evaluate(sentence)
    
        print('Input: %s' % (sentence))
        print('Predicted translation: {}'.format(result))
    
    
    def resore_last_checkpoint(self):
        self.checkpoint.restore(tf.train.latest_checkpoint(self.checkpoint_dir))

In [17]:
checkpoint_dir = './training_nmt_checkpoints'

In [18]:
translator = MyTranslator(dataset, vocab_inp_size, vocab_tar_size,
                          embedding_dim, units,
                          BATCH_SIZE, steps_per_epoch, checkpoint_dir)

In [19]:
translator.train_model(3)

Epoch 1 Batch 0 Loss 4.7787
Epoch 1 Batch 100 Loss 2.0838
Epoch 1 Batch 200 Loss 1.8292
Epoch 1 Batch 300 Loss 1.6489
Epoch 1 Batch 400 Loss 1.4714
Epoch 1 Batch 500 Loss 1.4439
Epoch 1 Batch 600 Loss 1.3974
Epoch 1 Batch 700 Loss 1.2451
Epoch 1 Batch 800 Loss 1.0957
Epoch 1 Batch 900 Loss 1.0997
Epoch 1 Batch 1000 Loss 1.1400
Epoch 1 Batch 1100 Loss 0.9178
Epoch 1 Batch 1200 Loss 1.0013
Epoch 1 Loss 1.4209
Time taken for 1 epoch 1552.2580831050873 sec

Epoch 2 Batch 0 Loss 0.8328
Epoch 2 Batch 100 Loss 0.8204
Epoch 2 Batch 200 Loss 0.8362
Epoch 2 Batch 300 Loss 0.8167
Epoch 2 Batch 400 Loss 0.7195
Epoch 2 Batch 500 Loss 0.6179
Epoch 2 Batch 600 Loss 0.6499
Epoch 2 Batch 700 Loss 0.6901
Epoch 2 Batch 800 Loss 0.5162
Epoch 2 Batch 900 Loss 0.5385
Epoch 2 Batch 1000 Loss 0.6719
Epoch 2 Batch 1100 Loss 0.5737
Epoch 2 Batch 1200 Loss 0.6441
Epoch 2 Loss 0.6780
Time taken for 1 epoch 1521.5289402008057 sec

Epoch 3 Batch 0 Loss 0.3196
Epoch 3 Batch 100 Loss 0.3934
Epoch 3 Batch 200 Loss 0.3

In [20]:
translator.translate('Я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . i'm sick . <end> 


### Модель с механизмом внимания

#### Энкодер

In [21]:
class EncoderAttention(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(EncoderAttention, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

#### Декодер

In [22]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

In [23]:
class DecoderAttention(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(DecoderAttention, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.attention = BahdanauAttention(self.dec_units)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights

#### Модель

In [24]:
class MyTranslatorAttention(MyTranslator):
    def __init__(self, dataset, vocab_inp_size, vocab_tar_size,
                 embedding_dim, units,
                 batch_size, steps_per_epoch, checkpoint_dir):
        super(MyTranslatorAttention, self).__init__(dataset, vocab_inp_size, vocab_tar_size,
                                                    embedding_dim, units,
                                                    batch_size, steps_per_epoch, checkpoint_dir)
        
        self.encoder = EncoderAttention(vocab_inp_size, embedding_dim, units, batch_size)
        self.decoder = DecoderAttention(vocab_tar_size, embedding_dim, units, batch_size)
        
        self.checkpoint = tf.train.Checkpoint(optimizer=self.optimizer,
                                              encoder=self.encoder,
                                              decoder=self.decoder)

    def train_step(self, inp, targ, enc_hidden):
        loss = 0
    
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = self.encoder(inp, enc_hidden)
        
            dec_hidden = enc_hidden
        
            dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * self.batch_size, 1)
        
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = self.decoder(dec_input, dec_hidden, enc_output)
            
                loss += self.loss_function(targ[:, t], predictions)

                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
    
        batch_loss = (loss / int(targ.shape[1]))
    
        variables = self.encoder.trainable_variables + self.decoder.trainable_variables
    
        gradients = tape.gradient(loss, variables)
    
        self.optimizer.apply_gradients(zip(gradients, variables))
    
        return batch_loss
    
    
    def evaluate(self, sentence):
        attention_plot = np.zeros((max_length_targ, max_length_inp))
    
        sentence = preprocess_sentence(sentence)
    
        inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
        inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                               maxlen=max_length_inp,
                                                               padding='post')
        inputs = tf.convert_to_tensor(inputs)
    
        result = ''
    
        hidden = [tf.zeros((1, self.units))]
        enc_out, enc_hidden = self.encoder(inputs, hidden)
    
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    
        for t in range(max_length_targ):
            predictions, dec_hidden, attention_weights = self.decoder(dec_input,
                                                                      dec_hidden,
                                                                      enc_out)
    
            # storing the attention weights to plot later on
            attention_weights = tf.reshape(attention_weights, (-1, ))
            attention_plot[t] = attention_weights.numpy()
    
            predicted_id = tf.argmax(predictions[0]).numpy()
    
            result += targ_lang.index_word[predicted_id] + ' '
    
            if targ_lang.index_word[predicted_id] == '<end>':
                return result, sentence, attention_plot
    
            # the predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)
    
        return result, sentence, attention_plot
    
    
    # function for plotting the attention weights
    def plot_attention(self, attention, sentence, predicted_sentence):
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(1, 1, 1)
        ax.matshow(attention, cmap='viridis')
    
        fontdict = {'fontsize': 14}
    
        ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
        ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)
    
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
        plt.show()
    
    
    def translate(self, sentence, with_plot=False):
        result, sentence, attention_plot = self.evaluate(sentence)
    
        print('Input: %s' % (sentence))
        print('Predicted translation: {}'.format(result))
    
        if with_plot:
            attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
            self.plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [25]:
checkpoint_dir_att = './training_nmt_att_checkpoints'

In [26]:
translator_att = MyTranslatorAttention(dataset, vocab_inp_size, vocab_tar_size,
                                       embedding_dim, units,
                                       BATCH_SIZE, steps_per_epoch, checkpoint_dir_att)

In [27]:
translator_att.train_model(3)

Epoch 1 Batch 0 Loss 4.8292
Epoch 1 Batch 100 Loss 2.1147
Epoch 1 Batch 200 Loss 1.8275
Epoch 1 Batch 300 Loss 1.7360
Epoch 1 Batch 400 Loss 1.5941
Epoch 1 Batch 500 Loss 1.4922
Epoch 1 Batch 600 Loss 1.3902
Epoch 1 Batch 700 Loss 1.2844
Epoch 1 Batch 800 Loss 1.2987
Epoch 1 Batch 900 Loss 1.1461
Epoch 1 Batch 1000 Loss 0.9689
Epoch 1 Batch 1100 Loss 0.7849
Epoch 1 Batch 1200 Loss 0.8259
Epoch 1 Loss 1.4638
Time taken for 1 epoch 2431.970602750778 sec

Epoch 2 Batch 0 Loss 0.6358
Epoch 2 Batch 100 Loss 0.7877
Epoch 2 Batch 200 Loss 0.7144
Epoch 2 Batch 300 Loss 0.7170
Epoch 2 Batch 400 Loss 0.6675
Epoch 2 Batch 500 Loss 0.6850
Epoch 2 Batch 600 Loss 0.4915
Epoch 2 Batch 700 Loss 0.5495
Epoch 2 Batch 800 Loss 0.4902
Epoch 2 Batch 900 Loss 0.4943
Epoch 2 Batch 1000 Loss 0.4090
Epoch 2 Batch 1100 Loss 0.5227
Epoch 2 Batch 1200 Loss 0.4902
Epoch 2 Loss 0.6064
Time taken for 1 epoch 2269.739987373352 sec

Epoch 3 Batch 0 Loss 0.3090
Epoch 3 Batch 100 Loss 0.4225
Epoch 3 Batch 200 Loss 0.322

In [28]:
translator_att.translate('Здесь хорошо.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's good here . <end> 


### Оценка качества перевода моделей

In [29]:
import pandas as pd

In [30]:
df = pd.DataFrame([len(str_val[str_val != 0]) for str_val in input_tensor_val], columns=['num_tokens'])
print(df['num_tokens'].unique())
df['index'] = df.index
df.head()

[ 7  8  6  5  9 10  4 11 13 12]


,num_tokens,index
0,7,0
1,7,1
2,8,2
3,6,3
4,6,4


In [33]:
for i in range(4, 14):
    indexes = df.loc[df['num_tokens']==i].sample(
        2, replace=True)['index'].values if len(df.loc[df['num_tokens']==i]) > 0 else []
    print(f'Tokens count: {i-2}')
    print(f'-----------------------------------------------------')
    for index in indexes:
        text_in = " ".join([inp_lang.index_word[t] if t > 2
                            else '' for t in input_tensor_val[index][input_tensor_val[index] != 0]])
        text_targ = " ".join([targ_lang.index_word[t] if t > 2
                              else '' for t in target_tensor_val[index][target_tensor_val[index] != 0]])
        print(f'Input: {text_in}')
        print(f'Target: {text_targ}')
        print(f'---------------------------------')
        translator.translate(text_in)
        translator_att.translate(text_in)
        print()
    print(f'-----------------------------------------------------')
    print()

Tokens count: 2
-----------------------------------------------------
Input:  свали ! 
Target:  get lost ! 
---------------------------------
Input: <start> свали ! <end>
Predicted translation: get dressed ! <end> 
Input: <start> свали ! <end>
Predicted translation: get it ! <end> 

Input:  перестань ! 
Target:  stop it . 
---------------------------------
Input: <start> перестань ! <end>
Predicted translation: stop ! <end> 
Input: <start> перестань ! <end>
Predicted translation: stop it . <end> 

-----------------------------------------------------

Tokens count: 3
-----------------------------------------------------
Input:  берите вс . 
Target:  get everything . 
---------------------------------
Input: <start> берите вс . <end>
Predicted translation: get everything . <end> 
Input: <start> берите вс . <end>
Predicted translation: take everything . <end> 

Input:  это сработало . 
Target:  that worked . 
---------------------------------
Input: <start> это сработало . <end>
Predicte

__ВЫВОДЫ:__ Обе модели демонстрируют примерно одинаковое качество перевода. Модель с механизмом внимания вероятно не проявляет своё превосходство, так как тексты относительно короткие и количество эпох обучения моделей было очень мало (существенное время одной эпохи обучения моделей, ~25 минут для первой и ~40 минут для второй модели).